In [1]:
%load_ext autoreload
%autoreload 2

# Saving Crawler Results 

Right now we're just throwing away the version messages we receive from our peers -- which contain interesting information about the node sofware they are using. We also don't keep any track record of the addresses we've successfully visited. That's essential if we want to make any claims that "The Bitcoin network contains at least N nodes".

I propose we save this information in a SQLite database. If you've never used SQL before, I suggest you do [this tutorial](http://www.sqlitetutorial.net/). If you'd like to dive in a little deeper, consider [Codecademy's course](https://www.codecademy.com/learn/learn-sql) or this collection of more theory-heavy ["database mini-courses"](https://lagunita.stanford.edu/courses/DB/2014/SelfPaced/about) offered by Stanford.

SQLite is a relational database that lives in a single file. Therefore, it's pretty easy to work with and quite portable -- you can even email the database file if you like!

And since SQLite is perhaps the most deployed piece of software in the world (it's on your phone, you laptop, your TV, your refrigerator, etc) there exists a lot of nice tooling for it. One great tool is [SQLiteBrowser](https://sqlitebrowser.org). Please install it. We'll use it to query the data generated by our crawler.

## SQLite Demo

Here's how to do the two most important operations:
* Add entries to the database with `INSERT`
* Query the database with `SELECT`

In [2]:
import sqlite3

# Connect to database
con = sqlite3.connect('test.db')

In [3]:
# Create table
con.execute("CREATE TABLE IF NOT EXISTS cars(id INT, name TEXT, price INT)")

# Insert entries into "cars" table (one by one)
con.execute("INSERT INTO cars VALUES(1,'Audi',52642)")
con.execute("INSERT INTO cars VALUES(2,'Mercedes',57127)")
con.execute("INSERT INTO cars VALUES(3,'Skoda',9000)")
con.execute("INSERT INTO cars VALUES(4,'Volvo',29000)")
con.execute("INSERT INTO cars VALUES(5,'Bentley',350000)")
con.execute("INSERT INTO cars VALUES(6,'Citroen',21000)")
con.execute("INSERT INTO cars VALUES(7,'Hummer',41400)")
con.execute("INSERT INTO cars VALUES(8,'Volkswagen',21600)")

# Fetch contents of the "cars" table
con.execute("SELECT * FROM cars").fetchall()

[(1, 'Audi', 52642),
 (2, 'Mercedes', 57127),
 (3, 'Skoda', 9000),
 (4, 'Volvo', 29000),
 (5, 'Bentley', 350000),
 (6, 'Citroen', 21000),
 (7, 'Hummer', 41400),
 (8, 'Volkswagen', 21600)]

In [4]:
# conn.execute can handle be parameterized

# with iterables of iterables
con.execute('INSERT INTO cars VALUES (?, ?, ?)', [9, "Chevy", 22000])

# with iterables of dictionaries
con.execute('INSERT INTO cars VALUES (:id, :name, :price)',
            {"id": 10, "name": "Honda", "price": 35000})

# Fetch contents of the "cars" table
con.execute("SELECT * FROM cars").fetchall()

[(1, 'Audi', 52642),
 (2, 'Mercedes', 57127),
 (3, 'Skoda', 9000),
 (4, 'Volvo', 29000),
 (5, 'Bentley', 350000),
 (6, 'Citroen', 21000),
 (7, 'Hummer', 41400),
 (8, 'Volkswagen', 21600),
 (9, 'Chevy', 22000),
 (10, 'Honda', 35000)]

In [5]:
# Re-create the table
con.execute('DELETE FROM cars')
print("Contents before executemany: ", con.execute("SELECT * FROM cars").fetchall())

# Insert entries into "cars" table (many at once)
args = [(1, 'Audi', 52642),
         (2, 'Mercedes', 57127),
         (3, 'Skoda', 9000),
         (4, 'Volvo', 29000),
         (5, 'Bentley', 350000),
         (6, 'Citroen', 21000),
         (7, 'Hummer', 41400),
         (8, 'Volkswagen', 21600)]
con.executemany("INSERT INTO cars VALUES(?, ?, ?)", args)
print("Contents after executemany: ", con.execute("SELECT * FROM cars").fetchall())

Contents before executemany:  []
Contents after executemany:  [(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600)]


In [6]:
# Query the expensive cars

price = 50000
rows = con.execute("SELECT * FROM cars WHERE price > :price", {'price': 50000}).fetchall()
print(rows)

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (5, 'Bentley', 350000)]


In [7]:
# Close the connection
con.close()

# Nodes Table

Just like we made a pretend "cars" table above, let's make an "nodes" and "connections" table for our crawler.

In [8]:
def execute(query, args={}):
    with sqlite3.connect('test.db') as conn:
        return conn.execute(query, args)

In [9]:
create_nodes_table_query = """
CREATE TABLE IF NOT EXISTS nodes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ip TEXT,
    port INT
)
"""
execute(create_nodes_table_query)

In [10]:
def insert_node(query_args):
    query = """
    INSERT INTO nodes (
        ip, port
    ) VALUES (
        :ip, :port
    )
    """
    return execute(query, query_args)

node = {
    'ip': '55.55.55.55',
    'port': 8333,
}

insert_node(node)
execute('SELECT * FROM nodes').fetchall()

IntegrityError: UNIQUE constraint failed: nodes.ip, nodes.port

In [11]:
# But if we call it twice we get duplicates ...
insert_node(node)
execute('SELECT * FROM nodes').fetchall()

IntegrityError: UNIQUE constraint failed: nodes.ip, nodes.port

In [12]:
# Set a UNIQUE constraint on the pair ip  port
create_nodes_table_query = """
CREATE TABLE IF NOT EXISTS nodes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ip TEXT,
    port INT,
    UNIQUE(ip, port)
)
"""

In [13]:
# Recreate the table
execute('DROP TABLE IF EXISTS nodes')
execute(create_nodes_table_query)

In [14]:
# Insert duplicates. Error raised when we insert duplicate ...
insert_node(node)

In [15]:
insert_node(node)

IntegrityError: UNIQUE constraint failed: nodes.ip, nodes.port

In [16]:
# First one makes it to database, however
execute('select * from nodes').fetchall()

[(1, '55.55.55.55', 8333)]

In [17]:
# Let's just ignore duplicates instead of treating them as errors:

# "INSERT INTO nodes" -> "INSERT OR IGNORE INTO"
def insert_node(query_args):
    query = """
    INSERT OR IGNORE INTO nodes (
        ip, port
    ) VALUES (
        :ip, :port
    )
    """
    return execute(query, query_args)

# Reset "nodes" table and insert duplicates
execute('DELETE FROM nodes')
insert_node(node)
insert_node(node)

# First one makes it to database, second one ignored!
execute('select * from nodes').fetchall()

[(2, '55.55.55.55', 8333)]

In [20]:
# Dictionaries are more ergonomic than tuples.
# This teaches SQLite to return dictionaries instead of tuples

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def execute(query, args={}, row_factory=None):
    with sqlite3.connect('test.db') as conn:
        if row_factory:
            conn.row_factory = row_factory
        return conn.execute(query, args)

print('Without dict_factory:', execute('select * from nodes').fetchone())
print('With dict_factory:', execute('select * from nodes', 
                            row_factory=dict_factory).fetchone())

Without dict_factory: (2, '55.55.55.55', 8333)
With dict_factory: {'id': 2, 'ip': '55.55.55.55', 'port': 8333}


In [21]:
from crawler import Node  # FIXME

# Have SQLite return Node instances instead of dictionaries!
def node_factory(cursor, row):
    return Node(**dict_factory(cursor, row))

print('With node_factory:', execute('select * from nodes', 
                            row_factory=node_factory).fetchone())

With node_factory: <crawler.Node object at 0x7f2762b3cef0>


## Connections Table

This table will contain:
* id
* Foreign Key w/ nodes table
* Unix timestamps for beginning of the handshake
* Everything in `version` messages

In [22]:
create_connections_table_query = """
CREATE TABLE IF NOT EXISTS connections (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    start INT,
    services INT,
    sender_timestamp INT,
    receiver_services INT,
    receiver_ip TEXT,
    receiver_port INT,
    sender_services INT,
    sender_ip TEXT,
    sender_port INT,
    nonce TEXT,
    user_agent TEXT,
    latest_block INT,
    relay INT,
    node_id INT,
    FOREIGN KEY(node_id) REFERENCES nodes(id)
)
"""

In [32]:
execute(create_connections_table_query)

In [33]:
def insert_connection(query_args):
    query = """
    INSERT INTO connections 
        (start, services, sender_timestamp, receiver_services, receiver_ip, receiver_port, sender_services, sender_ip, sender_port, nonce, user_agent, latest_block, relay, node_id)
    VALUES
        (:start, :services, :sender_timestamp, 
         :receiver_services, :receiver_ip, :receiver_port, :sender_services, :sender_ip, 
         :sender_port, :nonce, :user_agent, :latest_block, :relay, :node_id)
    """
    return execute(query, query_args)

In [34]:
import time

version_payload = {
    'version': 70015,
    'services': 1,
    'sender_timestamp': 1553189779,
    'receiver_services': 1,
    'receiver_ip': '0.0.0.0',
    'receiver_port': 8333,
    'sender_services': 1,
    'sender_ip': '0.0.0.0',
    'sender_port': 36128,
    'nonce': 15042168689231199477,
    'user_agent': b'/buidl-bootcamp/',
    'latest_block': 500000,
    'relay': 1,
    'ip': '0.0.0.0',
    'port': 8333}

args = version_payload.copy()

args['start'] = time.time() - 1

# Associate this "connection" with a "node" using the node's unique "id"
node = execute('select * from nodes', row_factory=node_factory).fetchone()
args['node_id'] = node.id

insert_connection(args)

OverflowError: Python int too large to convert to SQLite INTEGER

In [35]:
# nonce is too big to be SQLite INT, so let's just treat it as TEXT ...
args['nonce'] = str(args['nonce'])
insert_connection(args)

In [36]:
execute("select * from connections").fetchall()

[(1,
  1554529588.3563123,
  1,
  1553189779,
  1,
  '0.0.0.0',
  8333,
  1,
  '0.0.0.0',
  36128,
  '15042168689231199477',
  b'/buidl-bootcamp/',
  500000,
  1,
  2),
 (2,
  1554529932.1058066,
  1,
  1553189779,
  1,
  '0.0.0.0',
  8333,
  1,
  '0.0.0.0',
  36128,
  '15042168689231199477',
  b'/buidl-bootcamp/',
  500000,
  1,
  2)]

In [37]:
# dict_factory makes life much better ...
execute("select * from connections", row_factory=dict_factory).fetchall()

[{'id': 1,
  'start': 1554529588.3563123,
  'services': 1,
  'sender_timestamp': 1553189779,
  'receiver_services': 1,
  'receiver_ip': '0.0.0.0',
  'receiver_port': 8333,
  'sender_services': 1,
  'sender_ip': '0.0.0.0',
  'sender_port': 36128,
  'nonce': '15042168689231199477',
  'user_agent': b'/buidl-bootcamp/',
  'latest_block': 500000,
  'relay': 1,
  'node_id': 2},
 {'id': 2,
  'start': 1554529932.1058066,
  'services': 1,
  'sender_timestamp': 1553189779,
  'receiver_services': 1,
  'receiver_ip': '0.0.0.0',
  'receiver_port': 8333,
  'sender_services': 1,
  'sender_ip': '0.0.0.0',
  'sender_port': 36128,
  'nonce': '15042168689231199477',
  'user_agent': b'/buidl-bootcamp/',
  'latest_block': 500000,
  'relay': 1,
  'node_id': 2}]